# Loan Default Prediction - Modeling

For the Modeling we want to showcase the following machine learning tools:

<b>Keras/TensorFlow:</b> Keras is user-friendly and works well for deep learning. Our dataset is large enough and slightly complex, a neural network might capture the underlying patterns well. We will start with a simple feedforward neural network and gradually increase complexity if needed.


<b>PyTorch:</b> If possible we may look into Pytorch as it's great for custom model architectures and provides us with flexibility. This is a bit more challenging than than Keras but it is powerful. We begin with a basic architecture and explore more complex layers and structures if needed. 

At the end we will compare the two. 


#### Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix



In [3]:
df = pd.read_csv("../data/loan_data_cleaned.csv", index_col=0)

df.head()

,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Education,EmploymentType,MaritalStatus,HasMortgage,HasDependents,LoanPurpose,HasCoSigner,Default
Age,,,,,,,,,,,,,,,,
56,85994,50587,520,80,4,15.23,36,0.44,0,0,0,1,1,4,1,0
69,50432,124440,458,15,1,4.81,60,0.68,2,0,1,0,0,4,1,0
46,84208,129188,451,26,3,21.17,24,0.31,2,3,0,1,1,0,0,1
32,31713,44799,743,0,3,7.07,24,0.23,1,0,1,0,0,1,0,0
60,20437,9139,633,8,4,6.51,48,0.73,0,3,0,0,1,0,0,0


### Train-Test Split

In [4]:
# we are doing an 80/20 split

X = df.drop('Default', axis=1)  
y = df['Default']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)

## Neural Network with Keras/TensorFlow
We'll start by building a simple feedforward neural network (multilayer perceptron) for classification.

For our model we will have an input layer, two hidden layers (64 and 32 units), and an output layer. We will use ReLU for hidden layers and Sigmoid for the output layer since it’s a binary classification problem.

We will train the model with 10 epochs and adjust depending on the performance. (How many times the model will go through the training set)

Note:
Batch controls how many samples are processed before the model’s internal parameters are updated.
validation_split=0.2: This will use 20% of the training data for validation during training, which helps monitor the model's performance on unseen data.

In [5]:

model = Sequential()


model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))  

model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=32)


y_pred_prob = model.predict(X_test)
y_pred_class = (y_pred_prob > 0.5).astype("int32")

# Evaluate the model
auc = roc_auc_score(y_test, y_pred_prob)
print(f"AUC-ROC Score: {auc}")


Epoch 1/10
5107/5107 [==============================] - 12s 2ms/step - loss: 51.6466 - accuracy: 0.8022 - val_loss: 4.5954 - val_accuracy: 0.8844
Epoch 2/10
5107/5107 [==============================] - 11s 2ms/step - loss: 2.8266 - accuracy: 0.8455 - val_loss: 0.3581 - val_accuracy: 0.8845
Epoch 3/10
5107/5107 [==============================] - 10s 2ms/step - loss: 0.3604 - accuracy: 0.8835 - val_loss: 0.3579 - val_accuracy: 0.8845
Epoch 4/10
5107/5107 [==============================] - 10s 2ms/step - loss: 0.3598 - accuracy: 0.8836 - val_loss: 0.3582 - val_accuracy: 0.8845
Epoch 5/10
5107/5107 [==============================] - 10s 2ms/step - loss: 0.3599 - accuracy: 0.8836 - val_loss: 0.3581 - val_accuracy: 0.8845
Epoch 6/10
5107/5107 [==============================] - 10s 2ms/step - loss: 0.3599 - accuracy: 0.8836 - val_loss: 0.3579 - val_accuracy: 0.8845
Epoch 7/10
5107/5107 [==============================] - 11s 2ms/step - loss: 0.3599 - accuracy: 0.8836 - val_loss: 0.3579 - val_a

#### Keras/TensorFlow Evaluation



In [6]:

auc = roc_auc_score(y_test, y_pred_prob)
print(f"AUC-ROC Score: {auc}")

print("Classification Report:\n", classification_report(y_test, y_pred_class))

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_class))

AUC-ROC Score: 0.5
Classification Report:
               precision    recall  f1-score   support

           0       0.88      1.00      0.94     45167
           1       0.00      0.00      0.00      5903

    accuracy                           0.88     51070
   macro avg       0.44      0.50      0.47     51070
weighted avg       0.78      0.88      0.83     51070

Confusion Matrix:
 [[45167     0]
 [ 5903     0]]


C:\Users\darde\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\darde\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\darde\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### Observation
This shows that the model is performing poorly since it isn't distinguishing between defaults and non-defaults effectively.


This is because the model is biased towards the majority class (class 0)which is because are many more non-defaults than defaults. So in this case, the model learned to predict non-defaults every time, which leads to high accuracy but poor recall for the default cases.

To fix this we will using class weights, where we assign more weight to the minority class to encourage the model to pay more attention to it.

### Weight Classes

In [7]:
class_weights = compute_class_weight('balanced', classes=np.array([0, 1]), y=y_train)
class_weights_dict = {0: class_weights[0], 1: class_weights[1]}

print("Class Weights: ", class_weights_dict)

Class Weights:  {0: 0.5657796340713578, 1: 4.300568421052631}


Before making predictions on the test set, we are going to evaluate the model on the test set

In [8]:
history = model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=32, class_weight=class_weights_dict)
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

Epoch 1/10
5107/5107 [==============================] - 12s 2ms/step - loss: 0.6982 - accuracy: 0.4844 - val_loss: 0.6927 - val_accuracy: 0.8845
Epoch 2/10
5107/5107 [==============================] - 10s 2ms/step - loss: 0.6939 - accuracy: 0.5017 - val_loss: 0.6839 - val_accuracy: 0.8845
Epoch 3/10
5107/5107 [==============================] - 11s 2ms/step - loss: 0.6938 - accuracy: 0.4871 - val_loss: 0.7039 - val_accuracy: 0.1155
Epoch 4/10
5107/5107 [==============================] - 10s 2ms/step - loss: 0.6938 - accuracy: 0.4868 - val_loss: 0.7050 - val_accuracy: 0.1155
Epoch 5/10
5107/5107 [==============================] - 11s 2ms/step - loss: 0.6938 - accuracy: 0.4380 - val_loss: 0.6761 - val_accuracy: 0.8845
Epoch 6/10
5107/5107 [==============================] - 12s 2ms/step - loss: 0.6938 - accuracy: 0.5142 - val_loss: 0.7000 - val_accuracy: 0.1155
Epoch 7/10
5107/5107 [==============================] - 11s 2ms/step - loss: 0.6937 - accuracy: 0.4826 - val_loss: 0.7110 - val_ac

In [9]:
# Prediction
y_pred_prob = model.predict(X_test)
y_pred_class = (y_pred_prob > 0.5).astype("int32")

#Evaluation
auc = roc_auc_score(y_test, y_pred_prob)
print(f"AUC-ROC Score: {auc}")
print("Classification Report:\n", classification_report(y_test, y_pred_class))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_class))

1596/1596 [==============================] - 2s 1ms/step
AUC-ROC Score: 0.5
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00     45167
           1       0.12      1.00      0.21      5903

    accuracy                           0.12     51070
   macro avg       0.06      0.50      0.10     51070
weighted avg       0.01      0.12      0.02     51070

Confusion Matrix:
 [[    0 45167]
 [    0  5903]]


C:\Users\darde\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\darde\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\darde\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Observation
It looks like the class weights did not improve the model. The model is no better than random guessing

AUC-ROC Score: 0.5
Recall and Precision for Class 1 (defaults) are still 0:
Confusion Matrix: All instances are classified as non-defaults.

<b>Possible Reasons:</b>
SThe class imbalance might still be too strong, and even with class weights, the model struggles to predict defaults.

Our  architecture may not be well-suited for this problem.

Or our features might not be sufficient enough for the model.

<b>Next Steps</b>
We could look into increasing the class weights. 
Consider using SMOTE
Add more layers or neurons
Use Ensemble model such as XGBoost or Random Forest
Lower the threshold for classifying defaults (try 0.3 instead of 0.5)

### SMOTE 
We will try SMOTE (Synthetic Minority Over-sampling Technique), hopefully this could help balance the dataset better than class weights.

In [10]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

history = model.fit(X_train_resampled, y_train_resampled, validation_split=0.2, epochs=10, batch_size=32)


Epoch 1/10
9027/9027 [==============================] - 22s 2ms/step - loss: 0.6619 - accuracy: 0.6246 - val_loss: 0.9703 - val_accuracy: 0.0000e+00
Epoch 2/10
9027/9027 [==============================] - 22s 2ms/step - loss: 0.6616 - accuracy: 0.6250 - val_loss: 0.9823 - val_accuracy: 0.0000e+00
Epoch 3/10
9027/9027 [==============================] - 17s 2ms/step - loss: 0.6616 - accuracy: 0.6250 - val_loss: 0.9881 - val_accuracy: 0.0000e+00
Epoch 4/10
9027/9027 [==============================] - 18s 2ms/step - loss: 0.6617 - accuracy: 0.6250 - val_loss: 1.0024 - val_accuracy: 0.0000e+00
Epoch 5/10
9027/9027 [==============================] - 18s 2ms/step - loss: 0.6617 - accuracy: 0.6250 - val_loss: 0.9992 - val_accuracy: 0.0000e+00
Epoch 6/10
9027/9027 [==============================] - 19s 2ms/step - loss: 0.6616 - accuracy: 0.6250 - val_loss: 0.9875 - val_accuracy: 0.0000e+00
Epoch 7/10
9027/9027 [==============================] - 19s 2ms/step - loss: 0.6616 - accuracy: 0.6250 - v

In [11]:
#Evalute the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

1596/1596 [==============================] - 2s 1ms/step - loss: 0.5249 - accuracy: 0.8844
Test Loss: 0.5248675346374512
Test Accuracy: 0.8844135403633118


In [14]:
#We will be making predictions at a threshold of 0.5
y_pred_prob = model.predict(X_test)
y_pred_class = (y_pred_prob > 0.5).astype("int32") 


1596/1596 [==============================] - 2s 1ms/step


In [15]:
auc = roc_auc_score(y_test, y_pred_prob)
print(f"AUC-ROC Score: {auc}")

print("Classification Report:\n", classification_report(y_test, y_pred_class))

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_class))

AUC-ROC Score: 0.5
Classification Report:
               precision    recall  f1-score   support

           0       0.88      1.00      0.94     45167
           1       0.00      0.00      0.00      5903

    accuracy                           0.88     51070
   macro avg       0.44      0.50      0.47     51070
weighted avg       0.78      0.88      0.83     51070

Confusion Matrix:
 [[45167     0]
 [ 5903     0]]


C:\Users\darde\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\darde\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\darde\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Seems like we are having the same issue. AUC-ROC of 0.5 means that it's not able to differentiate between the two classes. There is definitely a class imbalance and it is leading the model to predict the majority class. 

So we will try this with a lower threshold. 

In [16]:
#Threshold of 0.4
y_pred_class = (y_pred_prob > 0.4).astype("int32")

In [17]:
auc = roc_auc_score(y_test, y_pred_prob)
print(f"AUC-ROC Score: {auc}")

print("Classification Report:\n", classification_report(y_test, y_pred_class))

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_class))

AUC-ROC Score: 0.5
Classification Report:
               precision    recall  f1-score   support

           0       0.88      1.00      0.94     45167
           1       0.00      0.00      0.00      5903

    accuracy                           0.88     51070
   macro avg       0.44      0.50      0.47     51070
weighted avg       0.78      0.88      0.83     51070

Confusion Matrix:
 [[45167     0]
 [ 5903     0]]


C:\Users\darde\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\darde\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\darde\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
#Threshold of 0.3
y_pred_class = (y_pred_prob > 0.3).astype("int32")

In [19]:
auc = roc_auc_score(y_test, y_pred_prob)
print(f"AUC-ROC Score: {auc}")

print("Classification Report:\n", classification_report(y_test, y_pred_class))

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_class))

AUC-ROC Score: 0.5
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00     45167
           1       0.12      1.00      0.21      5903

    accuracy                           0.12     51070
   macro avg       0.06      0.50      0.10     51070
weighted avg       0.01      0.12      0.02     51070

Confusion Matrix:
 [[    0 45167]
 [    0  5903]]


C:\Users\darde\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\darde\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\darde\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Fine-tune SMOTE

We are going to try to Fine-tune SMOTE to avoid overcompensating for the minority class.

In [20]:
smote = SMOTE(sampling_strategy=0.5, random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
history = model.fit(X_train_resampled, y_train_resampled, validation_split=0.2, epochs=10, batch_size=32)


Epoch 1/10
6770/6770 [==============================] - 15s 2ms/step - loss: 0.4517 - accuracy: 0.8333 - val_loss: 1.8137 - val_accuracy: 0.0000e+00
Epoch 2/10
6770/6770 [==============================] - 13s 2ms/step - loss: 0.4506 - accuracy: 0.8333 - val_loss: 1.7881 - val_accuracy: 0.0000e+00
Epoch 3/10
6770/6770 [==============================] - 13s 2ms/step - loss: 0.4506 - accuracy: 0.8333 - val_loss: 1.7820 - val_accuracy: 0.0000e+00
Epoch 4/10
6770/6770 [==============================] - 14s 2ms/step - loss: 0.4506 - accuracy: 0.8333 - val_loss: 1.8251 - val_accuracy: 0.0000e+00
Epoch 5/10
6770/6770 [==============================] - 13s 2ms/step - loss: 0.4506 - accuracy: 0.8333 - val_loss: 1.7638 - val_accuracy: 0.0000e+00
Epoch 6/10
6770/6770 [==============================] - 14s 2ms/step - loss: 0.4506 - accuracy: 0.8333 - val_loss: 1.8401 - val_accuracy: 0.0000e+00
Epoch 7/10
6770/6770 [==============================] - 14s 2ms/step - loss: 0.4506 - accuracy: 0.8333 - v

In [21]:
y_pred_prob = model.predict(X_test)
y_pred_class = (y_pred_prob > 0.5).astype("int32")

1596/1596 [==============================] - 2s 1ms/step


In [22]:
auc = roc_auc_score(y_test, y_pred_prob)
print(f"AUC-ROC Score: {auc}")
print("Classification Report:\n", classification_report(y_test, y_pred_class))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_class))

AUC-ROC Score: 0.5
Classification Report:
               precision    recall  f1-score   support

           0       0.88      1.00      0.94     45167
           1       0.00      0.00      0.00      5903

    accuracy                           0.88     51070
   macro avg       0.44      0.50      0.47     51070
weighted avg       0.78      0.88      0.83     51070

Confusion Matrix:
 [[45167     0]
 [ 5903     0]]


C:\Users\darde\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\darde\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\darde\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Final observation before moving on to Pytorch. 

##### Keras: 
Despite using SMOTE, the severe imbalance in the dataset is still affecting the model's ability to learn meaningful patterns for the minority class. It’s possible that the features in the dataset are not providing enough signal for the model to differentiate between defaults and non-defaults.

We can look into feature engineering, before doing that we will compare existing model dataset with Pytorch model. 

## Pytorch 

Pytorch does provide a bit more flexibility and control over the training process. This may help with handling class imbalance and improving model performance. 

We'll build a simple feedforward neural network (similar to Keras). Then add class weights to the loss function to balance the influence of the minority class.

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim

#### Define Model Architecture

We'll have 3 hidden layers with ReLU activation and a Sigmoid activation function (output) for binary classification.

In [24]:
class LoanDefaultNN(nn.Module):
    def __init__(self):
        super(LoanDefaultNN, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 1)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))  
        return x

#### Initialize the model, loss function and and optimizer
Since this is a binary classification problem, we use BCELoss as the loss function
Adam optimizer with a learning rate of 0.001 for gradient-based optimization

In [25]:
model = LoanDefaultNN()
criterion = nn.BCELoss() 
optimizer = optim.Adam(model.parameters(), lr=0.001)

#### Training
 The model is trained over 10 epochs, with backpropagation and optimization happening in each epoch

In [26]:
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)

X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

epochs = 10
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    
    loss.backward()
    optimizer.step()

    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

Epoch 1/10, Loss: 88.37362670898438
Epoch 2/10, Loss: 88.37362670898438
Epoch 3/10, Loss: 88.37362670898438
Epoch 4/10, Loss: 88.37362670898438
Epoch 5/10, Loss: 88.37362670898438
Epoch 6/10, Loss: 88.37362670898438
Epoch 7/10, Loss: 88.37362670898438
Epoch 8/10, Loss: 88.37362670898438
Epoch 9/10, Loss: 88.37362670898438
Epoch 10/10, Loss: 88.37362670898438


#### Predictions
We will first make predictions on the test set and Convert probabilities to binary predictions using a threshold.

In [27]:
model.eval()
with torch.no_grad():
    y_pred_prob = model(X_test_tensor).numpy()

In [28]:
y_pred_class = (y_pred_prob > 0.5).astype(int)

#### Evaulate the model performance

In [29]:
auc = roc_auc_score(y_test, y_pred_prob)
print(f"AUC-ROC Score: {auc}")

print("Classification Report:\n", classification_report(y_test, y_pred_class))

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_class))

AUC-ROC Score: 0.5
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00     45167
           1       0.12      1.00      0.21      5903

    accuracy                           0.12     51070
   macro avg       0.06      0.50      0.10     51070
weighted avg       0.01      0.12      0.02     51070

Confusion Matrix:
 [[    0 45167]
 [    0  5903]]


C:\Users\darde\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\darde\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\darde\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


It seems the PyTorch model is exhibiting a similar issue as before: predicting everything as class 1 (defaults), leading to poor overall performance.

AUC-ROC Score: 0.5: This score indicates that the model is performing no better than random guessing.
Recall for Class 1 is high (1.00), but precision is extremely low (0.12), meaning the model is predicting nearly every sample as class 1


#### Next Setup

We will try to add class weights to the loss function in PyTorch and model again.

Note:  class_counts.sum() / (2 * class_counts)  this step is normalizing the class weights

In [30]:
class_counts = torch.tensor([len(y_train[y_train == 0]), len(y_train[y_train == 1])], dtype=torch.float32)
class_weights = class_counts.sum() / (2 * class_counts)

#This is so we can convert the cass weights to a tensor
class_weights_tensor = torch.tensor([class_weights[0], class_weights[1]], dtype=torch.float32)


In [31]:
class LoanDefaultNN(nn.Module):
    def __init__(self):
        super(LoanDefaultNN, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 1)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))
        return x

model = LoanDefaultNN()


criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights_tensor[1])

optimizer = optim.Adam(model.parameters(), lr=0.001)

In [32]:
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)

X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

epochs = 10
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    
    loss.backward()
    optimizer.step()

    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')


Epoch 1/10, Loss: 1.3172080516815186
Epoch 2/10, Loss: 1.3172080516815186
Epoch 3/10, Loss: 1.3172080516815186
Epoch 4/10, Loss: 1.3172080516815186
Epoch 5/10, Loss: 1.3172080516815186
Epoch 6/10, Loss: 1.3172080516815186
Epoch 7/10, Loss: 1.3172080516815186
Epoch 8/10, Loss: 1.3172080516815186
Epoch 9/10, Loss: 1.3172080516815186
Epoch 10/10, Loss: 1.3172080516815186


In [35]:
# Test set Prediction 
model.eval()
with torch.no_grad():
    y_pred_prob = torch.sigmoid(model(X_test_tensor)).numpy()

y_pred_class = (y_pred_prob > 0.5).astype(int)

In [36]:
auc = roc_auc_score(y_test, y_pred_prob)
print(f"AUC-ROC Score: {auc}")

print("Classification Report:\n", classification_report(y_test, y_pred_class))

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_class))

AUC-ROC Score: 0.5
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00     45167
           1       0.12      1.00      0.21      5903

    accuracy                           0.12     51070
   macro avg       0.06      0.50      0.10     51070
weighted avg       0.01      0.12      0.02     51070

Confusion Matrix:
 [[    0 45167]
 [    0  5903]]


C:\Users\darde\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\darde\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\darde\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Observation
At this point, it’s clear that this approach with PyTorch and class weights isn’t working. The model still predicts all instances as class 1 (defaults), and the AUC-ROC score is again 0.5, indicating no improvement in distinguishing between defaults and non-defaults.

## Next step
We will take a step back and visit Feature Engineering. We will open a new notebook for feature engineering where we will look into possibly creating new features, transform skewed feature, etc. 